In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
path="/home/ftamagnan/dataset/"
name="total_metadata_training.npz"
data=dict(np.load(path+name))

data["random"]= np.random.rand(6729,3)


data['vae_embeddings']=data['vae_embeddings'][:,0:32]
for key in data.keys():
    print(data[key].shape,key)

(6729, 12) genre
(6729, 2) dataset
(6729, 32) vae_embeddings
(6729, 9) drums_pitches_used
(6729, 1) offbeat_notes
(6729, 9) count
(6729, 144) reduced_drums
(6729, 27) velocity_metadata
(6729, 3) random
(6729, 2, 1) fills
(6729, 1) bpm
(6729, 144) reduced_drums_velocity


In [3]:
def feature_selection(scaler=True,cv=True,list_list_label=[],penalty='l2',stats=True):

    
    
    
    for list_label in list_list_label:
            list_x=[]
            for key in data.keys():
                if key in list_label:
                    list_x.append(data[key])
            print(x.shape for x in list_x)
            X=np.concatenate(list_x,axis=1)
            y=data['fills'][:,1].reshape(-1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
            if scaler:
                scaler = StandardScaler()
                scaler.fit(X_train)
                X_train=scaler.transform(X_train)
                X_test=scaler.transform(X_test)
            
            if cv:
                clf = LogisticRegressionCV(cv=2, random_state=0,
                                   multi_class='ovr',penalty=penalty,solver='liblinear',max_iter=300,n_jobs=-1).fit(X_train, y_train)
            else:
                clf = LogisticRegression(random_state=0,C=100000000).fit(X_train, y_train)

#             y_pred=clf.predict(X_test)
            y_pred=(clf.predict_proba(X_test)>0.7)*1
            y_pred=y_pred[:,1]
            tn, fp, fn, tp=confusion_matrix(y_test, y_pred).ravel()
            if stats:
                print("__________Features used : "+str(list_label)+"_______")
                print("tn,fp,fn,tp = ",tn,fp,fn,tp)
                print("Accuracy = ",(tp+tn)/(tn+fp+fn+tp))
                print("Recall = ",(tp)/(fn+tp))
                print("Precision = ",(tp)/(fp+tp))

    return clf



# 1.Feature selection

In [4]:
list_list_label=[['vae_embeddings','drums_pitches_used','reduced_drums','velocity_metadata'],
               ['offbeat_notes','drums_pitches_used','velocity_metadata'],
               ['vae_embeddings'],
               ['offbeat_notes'],
               ['drums_pitches_used'],
               ['velocity_metadata'],
                 ['drums_pitches_used','velocity_metadata','vae_embeddings'],
                 ['vae_embeddings','velocity_metadata'],
                 ['random']
]

clf=feature_selection(scaler=True,cv=True,list_list_label=list_list_label)

<generator object feature_selection.<locals>.<genexpr> at 0x7fd1d0f3fd00>
__________Features used : ['vae_embeddings', 'drums_pitches_used', 'reduced_drums', 'velocity_metadata']_______
tn,fp,fn,tp =  1597 18 73 331
Accuracy =  0.9549281822684498
Recall =  0.8193069306930693
Precision =  0.9484240687679083
<generator object feature_selection.<locals>.<genexpr> at 0x7fd1d0f3fd00>
__________Features used : ['offbeat_notes', 'drums_pitches_used', 'velocity_metadata']_______
tn,fp,fn,tp =  1589 26 99 305
Accuracy =  0.9380881624566617
Recall =  0.754950495049505
Precision =  0.9214501510574018
<generator object feature_selection.<locals>.<genexpr> at 0x7fd1d0f3fd00>
__________Features used : ['vae_embeddings']_______
tn,fp,fn,tp =  1615 0 401 3
Accuracy =  0.8013868251609708
Recall =  0.007425742574257425
Precision =  1.0
<generator object feature_selection.<locals>.<genexpr> at 0x7fd1d0f3fd00>
__________Features used : ['offbeat_notes']_______
tn,fp,fn,tp =  1615 0 404 0
Accuracy =  0.799

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in long_scalars


__________Features used : ['drums_pitches_used']_______
tn,fp,fn,tp =  1586 29 215 189
Accuracy =  0.8791480931154037
Recall =  0.46782178217821785
Precision =  0.8669724770642202
<generator object feature_selection.<locals>.<genexpr> at 0x7fd1d0f3fd00>
__________Features used : ['velocity_metadata']_______
tn,fp,fn,tp =  1585 30 138 266
Accuracy =  0.9167904903417533
Recall =  0.6584158415841584
Precision =  0.8986486486486487
<generator object feature_selection.<locals>.<genexpr> at 0x7fd1d0f3fd00>
__________Features used : ['drums_pitches_used', 'velocity_metadata', 'vae_embeddings']_______
tn,fp,fn,tp =  1600 15 96 308
Accuracy =  0.9450222882615156
Recall =  0.7623762376237624
Precision =  0.9535603715170279
<generator object feature_selection.<locals>.<genexpr> at 0x7fd1d0f3fd00>
__________Features used : ['vae_embeddings', 'velocity_metadata']_______
tn,fp,fn,tp =  1597 18 101 303
Accuracy =  0.9410599306587419
Recall =  0.75
Precision =  0.9439252336448598
<generator object fea

In [5]:
list_list_label=[['vae_embeddings','offbeat_notes','velocity_metadata','drums_pitches_used']]
             

clf_l1=feature_selection(scaler=True,cv=True,list_list_label=list_list_label,penalty='l1',stats=False)
clf_l2=feature_selection(scaler=True,cv=True,list_list_label=list_list_label,penalty='l2',stats=False)

<generator object feature_selection.<locals>.<genexpr> at 0x7fd178a09f68>
<generator object feature_selection.<locals>.<genexpr> at 0x7fd1789e96d0>


In [6]:
name_pitches = ['bass drum','snare drum','closed hi-hat','open hi-hat','low tom','mid tom','high tom','crash cymbal','ride cymbal']
name_features=['min_velocity','max_velocity','std_velocity','max_velocity']

In [7]:
def stats_weights(clf):
    coef=clf.coef_
    coef=coef.reshape(-1)
    print("------VAE EMBEDDINGS-------")
    print(coef[0:32])

    print("------OFFBEATS NOTES-------")
    print(coef[32])

    print("------VELOCITY METRICS-------")
    for i,pitch in enumerate(name_pitches):
        for j,metric in enumerate(name_features):
            print(metric+' of '+pitch,coef[33+i+j])
        
    print("------PITCHES USED-------")
    for i,pitch in enumerate(name_pitches):
        print('use of '+pitch,coef[33+36+i])

# 2. Magnitude of weights with L1 reg

In [8]:
stats_weights(clf_l2)

------VAE EMBEDDINGS-------
[ 0.58310116  0.8992127   0.59963014  1.05243392 -0.88202531 -2.25120946
 -0.15965417 -1.17643139  0.21114501  0.82294222  0.00543798  1.51258404
 -0.09930574 -0.59186962 -0.27383607 -0.44670175  0.24297558 -0.5932663
 -0.22021114  0.63486609  0.14769127 -0.24089146 -0.59438018  0.28598805
 -0.80220408  0.34224337 -0.22065621 -1.35613132  0.07671929 -0.1874723
  0.43191542  0.80727847]
------OFFBEATS NOTES-------
-1.1978661365736374
------VELOCITY METRICS-------
min_velocity of bass drum -1.2728835322831005
max_velocity of bass drum 0.5926272583010685
std_velocity of bass drum -0.5496627480474168
max_velocity of bass drum 1.7097605548387023
min_velocity of snare drum 0.5926272583010685
max_velocity of snare drum -0.5496627480474168
std_velocity of snare drum 1.7097605548387023
max_velocity of snare drum -3.3150939134244717
min_velocity of closed hi-hat -0.5496627480474168
max_velocity of closed hi-hat 1.7097605548387023
std_velocity of closed hi-hat -3.31509

IndexError: index 69 is out of bounds for axis 0 with size 69

# 3.Magnitude of weights with L2 reg

In [ ]:
stats_weights(clf_l2)

In [9]:
from sklearn.externals import joblib
s = joblib.dump(clf_l2,'clf_fills.pkl')


# 4. Evaluate Generated model

In [11]:
list_list_label=[['vae_embeddings','count','drums_pitches_used']]


clf=feature_selection(scaler=True,cv=True,list_list_label=list_list_label)


<generator object feature_selection.<locals>.<genexpr> at 0x7fd1789e9780>
__________Features used : ['vae_embeddings', 'count', 'drums_pitches_used']_______
tn,fp,fn,tp =  1598 17 125 279
Accuracy =  0.9296681525507677
Recall =  0.6905940594059405
Precision =  0.9425675675675675
